<a href="https://colab.research.google.com/github/retico/cmepda_medphys/blob/master/L8_code/Lecture8_data_exploration_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data exploration and analysis**

Before we start the analysis of the brain features with ML techniques to search for a neuroimaging-based biomarker of the ASD condition, we explore the data cohort and create descriptive plots to represent it.

The data used in this demo is a table containing the brain features computed by means of the [FreeSurfer](https://surfer.nmr.mgh.harvard.edu/) segmentation software. A subsample of the large amount of features generated by Freesurfer for the [ABIDE I](http://fcon_1000.projects.nitrc.org/indi/abide/) data cohort is analyzed.  

We will have a quick look at [pandas](https://pandas.pydata.org/), one of the most used python data analysis libraries; we will use [matplotlib](https://matplotlib.org/), a comprehensive library for data visualization, and [seaborn](https://seaborn.pydata.org/), a high level API to matplotlib for statistical data visualization. For statistical data analysis we will introduce the [scipy](https://www.scipy.org/) library.

All these libraries are already installed on Colab. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

# Reading the dataset


We have to read a data table where each row corresponds to a different subject and the columns contain descriptive characteristics of each subjects (e.g. age, IQ, morphometric brain features).
A pandas DataFrames is definitly a suitable object to work with tabular data structures. It is basically a container for and exposes lots of methods to process tabular data. Tabular data are stored in pandas DataFrames, whereas data series (1-dim array) are stored in pandas Series.

Pandas offers plenty of readers out of the box.

In [ ]:
[x for x in dir(pd) if 'read' in x ]

Run this cell to mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/cmepda_medphys_dataset/FEATURES/Brain_MRI_FS_ABIDE/

Let us read a csv file as a pandas DataFrame. 


In [ ]:
dataset_file = "/content/drive/MyDrive/cmepda_medphys_dataset/FEATURES/Brain_MRI_FS_ABIDE/FS_features_ABIDE_males_someGlobals.csv"
# check and modify the path of the FS_features_ABIDE_males_someGlobals.csv file you downloaded in your drive

In [ ]:
df = pd.read_csv(dataset_file)
df.head()

In [ ]:
df.head(10)

In [ ]:
df.tail()

Check the size and shape of the DataFrame

In [ ]:
df.size

In [ ]:
df.shape

To access a sigle column:

In [ ]:
df.FILE_ID

In [ ]:
df['FILE_ID']

We can check that df is a DataFrame, whereas df.FILE_ID, which is a single column, is a pandas Series

In [ ]:
print(type(df), type(df.FILE_ID))

To access a single row

In [ ]:
df[0:1]

In [ ]:
df[df.FILE_ID=='Caltech_0051461']

We can add easily columns to a Dataframe, for example we can add a column containing data derived from the other columns

In [ ]:
df['dummy'] = df.DX_GROUP +1
df.head()

and we can delete it

In [ ]:
del df['dummy']

We can apply functions to the column values. 

For example, since it is hard to remember what DX_GROUP=1 actually means, we can make this column more readable.

First, we select the DX_GROUP column, then we apply a function to all its elements to convert the number to a meaningful label.

In [ ]:
df['DX_GROUP'] = df.DX_GROUP.apply(lambda x: 'Controls' if x==-1 else 'ASD')
df.head()

In [ ]:
df.DX_GROUP.unique()

We can make several operation/computations on the elements of the DataFrame: e.g. we can count the number of entries with a certain label:

In [ ]:
print(df[df.DX_GROUP=='ASD'].FILE_ID.count())
print(df[df.DX_GROUP=='Controls'].FILE_ID.count())

or we can compute the average age of a sample/subsample:

In [ ]:
df[df.DX_GROUP=='Controls'].AGE_AT_SCAN.mean()

Slicing: to select ranges of rows and/or columns it is possible to use either labels or indices.

In [ ]:
df.columns

using column labels (.loc)

In [ ]:
selected_feat = df.loc[:,'lh_MeanThickness':'rhCortexVol']
selected_feat.head()

using column indices (.iloc)

In [ ]:
df.iloc[:,5:9].head()

In [ ]:
df.iloc[1:4,:].head()

# Representing data



## Histograms

In [ ]:
df.hist('AGE_AT_SCAN')

In [ ]:
df.hist(['AGE_AT_SCAN','FIQ'])

By convention, in this dataset missing values in the FIQ column are indicated as either -9999 or 0

In [ ]:
df.FIQ[:20]

We can mask these entries:

In [ ]:
df[df.FIQ>0].hist('FIQ')

## Plots

Let's have a look at the DataFrame methods for data plotting

In [ ]:
[x for x in dir(df) if 'plot' in x ]

In [ ]:
df.AGE_AT_SCAN[:30].plot(kind='bar')

## Boxplots

Boxplot are very useful in analysis reports. We can for example visually check whether two cohorts are matched for some parameters (e.g. age).

In [ ]:
boxplot = df.boxplot(column=['AGE_AT_SCAN'], by='DX_GROUP', showfliers=False)
boxplot.set_title('Box plot of subject\'s age at scan')
boxplot.get_figure().suptitle('');

If we want, for example, to group subjects according to the acquisition site, we have to retrive this information in the DataFrame. We find out that the site name is a part of the FILE_ID.

We can use the .split method available for the strings, and use it on the DataFrame column elements by defining a suitable lambda function. 

In [ ]:
df.FILE_ID

In [ ]:
df.FILE_ID[0].split('_')[0]

We add the "Site" column to the DataFrame

In [ ]:
df['Site'] = df.FILE_ID.apply(lambda x: x.split('_')[0])
df.head()

We can make a boxplot representing the age values for each acquisition site.

In [ ]:
boxplot = df.boxplot(column=['AGE_AT_SCAN'], by='Site', showfliers=False)
boxplot.set_title('Box plot of subject\'s age at scan')
boxplot.get_figure().suptitle('');
boxplot.set_ylabel('Age [y]')

boxplot.set_xticklabels(labels=boxplot.get_xticklabels(), rotation=50);

The [seaborn](https://seaborn.pydata.org/) API interoperates well with pandas DataFrame and allows us to refer to columns by label

In [ ]:
import seaborn as sns
sns_boxplot = sns.boxplot(x='Site', y='AGE_AT_SCAN', data=df)
sns_boxplot.set_xticklabels(labels=boxplot.get_xticklabels(), rotation=50);
sns_boxplot.grid()

We can visually compare the measured values across different sites.

In [ ]:
df.columns

In [ ]:
import seaborn as sns
sns_boxplot = sns.boxplot(x='Site', y='lh_MeanThickness', data=df)
sns_boxplot.set_xticklabels(labels=boxplot.get_xticklabels(), rotation=50);
sns_boxplot.grid()

## Grouping

Data can be grouped by feature and visualized according to a given aggregation function

In [ ]:
site_counts = df.groupby('Site').count()
site_counts

In [ ]:
site_counts = df.groupby('Site')['FILE_ID'].count()
site_counts

It is quite easy to obtain a bar plot from a pandas Series

In [ ]:
site_counts.plot(kind='bar', title='Number of subjects per site')


Now let's try to create a stacked bar plot showing the DX_GROUP, i.e. how many ASD and how many controls are available at each single site

In [ ]:
stack = df.groupby(['Site', 'DX_GROUP'])['FILE_ID'].count()
stack

In [ ]:
unstacked = stack.unstack('DX_GROUP')
unstacked

In [ ]:
unstacked.plot(kind='bar', stacked=False, title='Number of subjects per site')

We can select a number of columns (data slicing)

In [ ]:
df.columns

In [ ]:
selected_feat = df.loc[:,'lh_MeanThickness':'rhCortexVol']
selected_feat.head()

Let's see how our selection looks like with the seaborn pairplot!

In [ ]:
sns.pairplot(selected_feat)

# Basic data analysis

To carry out basic data analysis, we use the [SciPy](https://www.scipy.org/scipylib/index.html) library, which provides many user-friendly and efficient numerical routines, such as routines for numerical integration, interpolation, optimization, linear algebra, and statistics.

In [ ]:
import scipy.stats

## Finding outliers in the distributions

In [ ]:
df.columns

We select the 7 columns reporting the brain measures

In [ ]:
data = df.iloc[:,5:12]

We use the z-score as a criterion to determine the presence of outliers.

Z-score is defined as:

$z(x) = \frac{x - \bar{x}}{\sigma}$. 

Data with a z-score above 3 (beyond 3$\sigma$ from the mean) are considered as outliers of the distribution.

In [ ]:
df_no_outliers=df[(abs(scipy.stats.zscore(data)) < 3).all(axis=1)]  
# .all Return whether all elements are True, potentially over an axis.
df_no_outliers.shape

In [ ]:
sns_boxplot = sns.boxplot(x='Site', y='AGE_AT_SCAN', data=df)
sns_boxplot.set_xticklabels(labels=boxplot.get_xticklabels(), rotation=50);
sns_boxplot.grid()

In [ ]:
sns_boxplot = sns.boxplot(x='Site', y='AGE_AT_SCAN', data=df_no_outliers)
sns_boxplot.set_xticklabels(labels=boxplot.get_xticklabels(), rotation=50);
sns_boxplot.grid()

## Two sample t-test

Is there any significant difference in the AGE_AT_SCAN and FIQ features between the two diagnostic categories?

In [ ]:
df_ASD = df[df.DX_GROUP == 'ASD']
df_CTR = df[df.DX_GROUP == 'Controls']

In [ ]:
ttest_res = scipy.stats.ttest_ind(df_ASD.AGE_AT_SCAN, df_CTR.AGE_AT_SCAN)
ttest_res

In [ ]:
ttest_res = scipy.stats.ttest_ind(df_ASD.FIQ, df_CTR.FIQ)
ttest_res

Let us define a new index of left-right asimmetry of the mean cortical thickness

In [ ]:
def LR(data):
  LR = data.lh_MeanThickness-data.rh_MeanThickness
  LR /= 0.5*(data.rh_MeanThickness+data.lh_MeanThickness)
  data['LR'] = LR
  return data

In [ ]:
data = LR(df)
data.head()

In [ ]:
color = data.DX_GROUP.apply(lambda x:'blue' if x == 'ASD' else 'red')
ax = data.plot(x='AGE_AT_SCAN', y='LR', kind='scatter', color=color);
ax.grid()

In [ ]:
data.DX_GROUP.unique()

In [ ]:
LR_ASD = data[data['DX_GROUP'] == 'ASD']['LR']
LR_CTR = data[data['DX_GROUP'] == 'Controls']['LR']

In [ ]:
scipy.stats.ttest_ind(LR_ASD, LR_CTR)

In [ ]:
boxplot = df.boxplot(column=['LR'], by='DX_GROUP', showfliers=False)
boxplot.set_title('Box plot of subject\'s LR asimmetry index')
boxplot.get_figure().suptitle('');
boxplot.set_ylabel('LR')

boxplot.set_xticklabels(labels=boxplot.get_xticklabels());

We can compute the effect size in terms of Cohen's *d* index. 

A population effect size based on means usually computed as:

$d = \frac{\mu_1-\mu_2}{\sigma}$




In [ ]:
d_cohen = (LR_ASD.mean()-LR_CTR.mean())/data['LR'].std()
d_cohen

## Correlations among variables

Let's have a look at the data from a correlation perspective: can we spot any relationship? 

In [ ]:
data.drop('SEX', axis=1).corr()

In [ ]:
sns.heatmap(data.drop('SEX', axis=1).corr());

In [ ]:
res = scipy.stats.pearsonr(data['rhCortexVol'], data['lhCortexVol'])
res

In [ ]:
res = scipy.stats.pearsonr(data['TotalGrayVol'], data['LR'])
res

# Conclusions

We've had an extremely quick overview of data exploration, visualization and statistical analysis methods.

To learn more for example about the possible alterations of the left-righ brain asymmetry in ASD you can  read the recent work by
Postema MC, *et al.*, [ENIGMA ASD](http://enigma.ini.usc.edu/ongoing/enigma-asd-working-group/) working group, [*Altered structural brain asymmetry in autism spectrum disorder in a study of 54 datasets*](https://www.nature.com/articles/s41467-019-13005-8), Nat Commun. 2019 10(1):4958. doi:
10.1038/s41467-019-13005-8.